In [6]:
import pandas as pd
from openai import OpenAI, Embedding
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import os


In [7]:
os.chdir("path/to/IR-With-Random-Deletion/")
df = pd.read_csv("11_20_2024_Articles.csv") 
#openai_client = OpenAI(api_key = 'insert-api-key')

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'path/to/IR-With-Random-Deletion/'

In [149]:
print(df.head())

           title                                                url  \
0  Yahoo Finance  https://finance.yahoo.com/news/veteran-fund-ma...   
1  Yahoo Finance  https://finance.yahoo.com/news/messenger-gets-...   
2  Yahoo Finance  https://finance.yahoo.com/news/social-media-ba...   
3  Yahoo Finance  https://finance.yahoo.com/news/snowflake-share...   
4  Yahoo Finance  https://finance.yahoo.com/news/baidu-bulls-ret...   

                          date  \
0   Wed, Nov 20, 2024, 3:33 PM   
1  Wed, Nov 20, 2024, 10:06 AM   
2   Wed, Nov 20, 2024, 5:00 PM   
3   Wed, Nov 20, 2024, 1:30 PM   
4   Wed, Nov 20, 2024, 4:28 PM   

                                             content  
0  We are experiencing some temporary issues. The...  
1  We are experiencing some temporary issues. The...  
2  We are experiencing some temporary issues. The...  
3  We are experiencing some temporary issues. The...  
4  We are experiencing some temporary issues. The...  


In [150]:
from openai import OpenAI, Embedding


def summarize_doc(openai_client, text):
    query = "Summarize this into less than 10 words"

    prompt = f"Context:\n{text}\n\nQuestion: {query}\nAnswer:"


    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
        {"role": "user", "content": prompt}
        ]
    )

    print(completion.choices[0].message)

In [151]:
def preprocess_text(text):
    """Preprocess a single text input (either document or query)."""
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')  # Add custom stop words if needed
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))  # Replace punctuation with spaces
    text_nopunct = text.translate(translator)  # Remove punctuation
    words_list = text_nopunct.strip().split()  # Tokenize into words
    filtered_words = [word.lower() for word in words_list if word.lower() not in stop_words and len(word) > 1]
    return filtered_words

def preprocess_df(df):
    """Preprocess a DataFrame column containing document content."""
    df["content"] = df["content"].apply(preprocess_text)
    return df

def preprocess_query(query):
    """Preprocess a query string."""
    return preprocess_text(query)

In [152]:
import bmp25
import importlib

# Reload the module after making changes
importlib.reload(bmp25)

# Initialize the BM25 instance

def rank_query_bm25(bm25, query, top_k=5, summarize_docs=True):
    # Display initial document details
    print("=" * 50)
    print("📄 Document Sample:")
    print(df.head())
    print("=" * 50)
    print(f"📏 Total Documents: {len(bm25_instance.doc_lengths)}")
    print(f"Preprocessed query: {query}")
    # Rank documents for a given query
    ranked_indices, scores = bm25_instance.rank(query)

    ranked_indices = ranked_indices[:top_k]
    scores = scores[:top_k]

    print("\n🏆 Ranking Results:")
    print("=" * 50)
    for i, (doc_index, score) in enumerate(zip(ranked_indices, scores), start=1):
        print(f"⭐ Rank {i}:")
        print(f"   - Document Index: {doc_index}")
        print(f"   - BM25 Score: {score:.4f}")
        doc_content = df.iloc[doc_index]["content"]
        if (summarize_docs):
            summmary = summarize_doc(openai_client, doc_content)
            print(f"   - Document Content:\n{summmary}")
        else:
            print(f"   - Document Content:\n{doc_content}")
        print("-" * 50)


In [153]:
query = "Why is Australia's monetary policy considered potent despite high household mortgage debt?" #190
query2 = "What is Palo Alto Networks' revenue forecast for the second quarter, and how does it compare with analyst expectations?" #34
query3= "How might institutional investments influence Bitcoin's price trajectory through 2025?" #11

query = query3

In [154]:
df = preprocess_df(df)
query = preprocess_query(query)

bm25_instance = bmp25.BM25(df["content"],k1=1.5, b=0.75)
rank_query_bm25(bm25_instance, query, summarize_docs=False)


📄 Document Sample:
           title                                                url  \
0  Yahoo Finance  https://finance.yahoo.com/news/veteran-fund-ma...   
1  Yahoo Finance  https://finance.yahoo.com/news/messenger-gets-...   
2  Yahoo Finance  https://finance.yahoo.com/news/social-media-ba...   
3  Yahoo Finance  https://finance.yahoo.com/news/snowflake-share...   
4  Yahoo Finance  https://finance.yahoo.com/news/baidu-bulls-ret...   

                          date  \
0   Wed, Nov 20, 2024, 3:33 PM   
1  Wed, Nov 20, 2024, 10:06 AM   
2   Wed, Nov 20, 2024, 5:00 PM   
3   Wed, Nov 20, 2024, 1:30 PM   
4   Wed, Nov 20, 2024, 4:28 PM   

                                             content  
0  [experiencing, temporary, issues, market, data...  
1  [experiencing, temporary, issues, market, data...  
2  [experiencing, temporary, issues, market, data...  
3  [experiencing, temporary, issues, market, data...  
4  [experiencing, temporary, issues, market, data...  
📏 Total Documents: 

In [155]:
count=0
for i in range(len(df)):
    article = df.iloc[i]["content"]
    if len(article) < 100:
        count+=1
        print(article)
print(count)

['experiencing', 'temporary', 'issues', 'market', 'data', 'page', 'currently', 'delayed', 'please', 'bear', 'us', 'address', 'restore', 'personalized', 'lists', 'singapore', 'reuters', 'dollar', 'bond', 'prices', 'adani', 'companies', 'fell', 'sharply', 'early', 'asia', 'trade', 'indian', 'conglomerate', 'billionaire', 'chairman', 'indicted', 'new', 'york', 'allegations', 'bribery', 'fraud', 'prices', 'adani', 'port', 'special', 'economic', 'zone', 'debt', 'maturing', 'august', '2027', 'fell', 'five', 'cents', 'dollar', 'according', 'lseg', 'data', 'adani', 'electricity', 'mumbai', 'debt', 'maturing', 'february', '2030', 'fell', 'nearly', 'eight', 'cents', 'dollar', 'bonds', 'issued', 'adani', 'transmission', 'also', 'notched', 'falls', 'larger', 'five', 'cents', 'reporting', 'tom', 'westbrook', 'editing', 'tom', 'hogue', 'sign', 'access', 'portfolio']
['experiencing', 'temporary', 'issues', 'market', 'data', 'page', 'currently', 'delayed', 'please', 'bear', 'us', 'address', 'restore',